### 为每个函数、类和模块编写文档字符串

In [42]:
# python 代码支持文档字符串和__doc__属性
def test():
    """test function"""
    a = 1

print(repr(test.__doc__))
help(test)

'test function'
Help on function test in module __main__:

test()
    test function



### 在重视精确度的场合，应该使用 decimal

In [38]:
# decimal 可以控制精度和舍入方式
from decimal import Decimal
import decimal
rate = Decimal('1.45')
seconds = Decimal('222')
cost = rate * seconds / Decimal('60')
print(cost)

rounded = cost.quantize(Decimal('0.01'), rounding=decimal.ROUND_UP)
print(rounded)

5.365
5.37


### 使用内置算法与数据结构

#### 双向队列：collections.deque

In [23]:
# 双向队列：collections.deque，从队列的头部或尾部插入或移除一个元素，只需消耗常数级别的时间
# 非常适合：先进先出队列 FIFO
# list：list在尾部插入或移除元素，是常数级别时间；但在list头部插入或移除元素，需要消耗线性级别时间
from collections import deque
fifo = deque()
fifo.append(1)
fifo.popleft()

0:00:00.000100
0:00:00.000088
0:00:00.000094
0:00:00.000100


#### 有序字典 collections.OrderedDict

In [24]:
# 标准的字典是无序的
# OrderedDict 能按照键的插入顺序，来保留键值对在字典中的次序
# 在OrderedDict上根据键来迭代时，其行为是确定的
from collections import OrderedDict

a = OrderedDict()
a['foo'] = 1
a['bar'] = 2
b = OrderedDict()
b['foo'] = 'red'
b['bar'] = 'blue'
for v1, v2 in zip(a.values(), b.values()):
    print(v1, v2)

1 red
2 blue


#### 带有默认值的字典 collections.defaultdict

In [25]:
# defaultdict: 如果字典里面没有待访问的键，就可以自动把某个默认值与这个键自动关联起来
from collections import defaultdict
states = defaultdict(int) # 使用内置的int函数来创建字典，默认值为0
states['a'] += 1
print(states)

defaultdict(<class 'int'>, {'a': 1})


#### 堆队列（优先级队列）heapq

In [28]:
# heaqp 操作所消耗的时间，与列表长度的对数成正比（优势）
# 对比：如果在普通的python列表上面执行相关操作，需要消耗线性级别的时间
from heapq import heappush, heappop
a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)
print(a) # 自动排好序，第一个元素总是最小的
print(heappop(a), heappop(a), heappop(a), heappop(a)) # 依次取出最小元素

[3, 4, 7, 5]
3 4 5 7


#### 二分查找 bisect.bisect_left

In [35]:
# bisect_left会返回待搜寻的值在序列中的插入点
# bisect_left 二分搜索算法的复杂度是对数级别的
from bisect import bisect_left, bisect_right
x = list(range(10**6))
i = bisect_left(x, 991234)
print(i)
x[100] = 100
x[101] = 100
i = bisect_left(x, 100)
print(i)
i = bisect_right(x, 100)
print(i)

991234
100
102


#### 与迭代器有关的工具：itertools

能够把迭代器连接起来
- chain：将多个迭代器按顺序连成一个迭代器
- cycle：无限地重复某个迭代器中的各个元素
- tee：把一个迭代器拆分成多个平行的迭代器
- zip_longest：与内置的zip函数相似，但是可以应对长度不同的迭代器

能够从迭代器中过滤元素的函数：
- islice：在不进行复制的前提下，根据索引值来切割迭代器
- takewhile：在判定函数为True的时候，从迭代器中逐个返回元素
- dropwhile：从判别函数初次为False的地方开始，逐个返回迭代器中的元素
- filterfalse：从迭代器中逐个返回能令判定函数为False的所有元素，与filter相反

能够把迭代器中元素组合起来的函数：
- product：根据迭代器中的元素计算笛卡尔积，并将其返回，可以用此来改写深度嵌套的列表推导操作
- permutations：用迭代器中的元素构建长度为N的各种有序排列，并将所有排队形式返回给调用者
- combinations：用迭代器中的元素构建长度为N的各种无序排列

### 应该用 datatime 模块来处理本地时间，而不是用time模块

In [6]:
# 协调世界时（Coordinated Universal Time UTC）是一个标准的时间表达式，与时区无关

# 把UTC格式的当前时间，转换为计算机所用的本地时间
from datetime import datetime, timezone
now = datetime(2022, 5, 22, 8, 56, 30)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

# 把本地时间转换为UTC格式的UNIX时间戳
from time import mktime
time_str = '2022-05-22 08:59:30'
time_format = '%Y-%m-%d %H:%M:%S'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = mktime(time_tuple)
print(utc_now)

2022-05-22 16:56:30+08:00
1653181170.0


### 用 copyreg 实现可靠的 pickle 操作

In [15]:
# pickle 模块能够将python对象序列化为字节流，也能反序列为python对象
# pickle 模型所产生的的序列化数据是一种不安全的格式，序列化数据实际上是一个程序， 描述了如何来创建原始的python对象
# pickle 模块只适合用来在彼此信任的程序之间，对相关对象执行序列化和反序列化操作
# json模块安全，序列化后的json数据，只是包含简单的描述信息

# 使用 copyreg为缺失的属性提供默认值
import copyreg
import pickle

class GameState:
    def __init__(self, level=0, lives=4, points=0):
        self.level = level
        self.lives = lives
        self.points = points

def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    return GameState(**kwargs)

copyreg.pickle(GameState, pickle_game_state) # copyreg模块来注册pickle_game_state函数

# 序列化和反序列化
state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

# 更新GameState，添加新的变量
class GameState:
    def __init__(self, level=0, lives=4, points=0, magic=5):
        self.level = level
        self.lives = lives
        self.points = points
        self.magic = magic

state_after = pickle.loads(serialized)
print(state_after.__dict__) # magic有了默认值，若不采用copyreg注册，则该项会丢失

{'level': 0, 'lives': 4, 'points': 1000}
{'level': 0, 'lives': 4, 'points': 1000, 'magic': 5}


In [16]:
# 用版本号来管理类：当更新后的对象删除了某些变量，则旧的对象反序列化会报错
def pickle_game_state(game_state):
    kwargs = game_state.__dict__
    kwargs['version'] = 2 # 添加版本标记
    return unpickle_game_state, (kwargs,)

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1) # 不存在该key时默认为1
    if version == 1:
        kwargs.pop('lives')
    return GameState(**kwargs)

# 更新版本， 删除 lives
class GameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

copyreg.pickle(GameState, pickle_game_state)

state_after = pickle.loads(serialized)
print(state_after.__dict__) # 旧版中有lives，新版的反序列化不会报错

{'level': 0, 'points': 1000, 'magic': 5}


In [18]:
# 固定引入路径： 当原来类的路径改变或改名后，旧的序列数据就无法反序列化
# 用copyreg来注册pickle_game_state，则序列化数据不再指向类，而是unpickle_game_state函数
# 因此要求unpickle_game_state不能发生改变
class GameState:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

class GameStateNew:
    def __init__(self, level=0, points=0, magic=5):
        self.level = level
        self.points = points
        self.magic = magic

def unpickle_game_state(kwargs):
    version = kwargs.pop('version', 1) # 不存在该key时默认为1
    if version == 1:
        kwargs.pop('lives')
    return GameStateNew(**kwargs)

copyreg.pickle(GameState, pickle_game_state)

state = GameState()
state.points += 1000
serialized = pickle.dumps(state)
state_after = pickle.loads(serialized)
print(state_after.__dict__)

{'level': 0, 'points': 1000, 'magic': 5}


###  考虑用 contextlib 和 with 语句来改写可复用的try/finally代码

In [7]:
# 用 with 语句可以免去编写 try/finally 结构所需的重复代码
# 通过contextmanager装饰器，来使得函数可用在with语句之中
# 定义一种情境管理器，来临时提升logging打印的信息级别
import logging
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield # with 块中的语句所要展开执行的地方，with 块所抛出的任何异常，都有yield表达式重新抛出
    finally:
        logger.setLevel(old_level)

def my_func():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')

with debug_logging(logging.DEBUG):
    print('in with')
    my_func()
print('out with')
my_func() # 只打印error，因为打印级别默认为warning

DEBUG:root:Some debug data
ERROR:root:Error log here
DEBUG:root:More debug data
ERROR:root:Error log here


in with
out with


In [8]:
# with as 语句，传给 with 语句的那个情境管理器，本身会可以返回一个对象
# 通过 as 来创建一个局部变量，指向返回的对象
# 例如，打印文件写入数据，with后自动关闭句柄
with open('./test.txt', 'w') as handle:
    handle.write('data')

In [11]:
# 在自定义的情境管理器中，只需要在yield后面加上一个对象，则返回as关键字所指定的目标变量
@contextmanager
def debug_logging(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)

with debug_logging(logging.DEBUG, 'my-log') as logger:
    print(logger)
    logger.debug('This is my message')
    logging.debug('This is not print') # 系统本身的logging的打印级别还是warning

print(logger) # 恢复成warning

DEBUG:my-log:This is my message


<Logger my-log (DEBUG)>
<Logger my-log (WARNING)>


### 用 functools.wraps 定义函数修饰器

In [9]:
# 装饰器：用来装饰函数，对于受到封装的原函数来说，装饰器能够在那个函数执行之前以及执行完毕后，
# 分别运行一些附加代码，这使得开发者可以在装饰器里访问并修改原函数的参数及返回值。
# 以实现约束语义（enforce semantics）、调试程序、注册函数等目标

# 例子：打印递归栈的每一个层级上所具备的参数和返回值
from functools import wraps

def trace(func):
    @wraps(func) # 必须加，保留原函数的某些标准python属性
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r'%
            (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """return the n-th fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n-2) + fibonacci(n-1))
# 等同于 fibonacci = trace(fibonacci)

print(fibonacci)
fibonacci(5)
print(help(fibonacci))

<function fibonacci at 0x7f7792d280d0>
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3
fibonacci((5,), {}) -> 5
Help on function fibonacci in module __main__:

fibonacci(n)
    return the n-th fibonacci number

None


### 用元类来验证子类

In [17]:
# 用元类来验证子类中的属性a的取值在[0,1]

class Meta(type): # 定义元类，继承于type
    def __new__(meta, name, bases, class_dict):
        print((meta, name, bases, class_dict))
        if bases != (object,): # 仅验证子类，不验证基类
            if not (0 <= class_dict['a'] <=1):
                raise ValueError('a must be [0, 1]')
        return type.__new__(meta, name, bases, class_dict)

class test(object, metaclass=Meta): # 定义基类
    a = None # 由子类指定

class subtest(test):
    a = -1

st = subtest() # 报错，元类验证了a

(<class '__main__.Meta'>, 'test', (<class 'object'>,), {'__module__': '__main__', '__qualname__': 'test', 'a': None})
(<class '__main__.Meta'>, 'subtest', (<class '__main__.test'>,), {'__module__': '__main__', '__qualname__': 'subtest', 'a': -1})


ValueError: a must be [0, 1]

### \_\_getattr\_\_ \_\_getattribute\_\_ __setattr__ （此类挂钩）实现按需生成的属性

In [8]:
# 若某个类定义了__getattr__,同时系统在该类对象的实例字典中找不到待查询的属性，则会调用这个方法
class test:
    def __init__(self):
        self.a = 5
    
    def __getattr__(self, name):
        value = 'Value for %s'% name
        setattr(self, name, value) # 创建变量
        return value

t = test()
print(f'before: {t.__dict__}')
print(f'foo: {t.foo}') # 调用了__getattr__
print(f'after: {t.__dict__}')

before: {'a': 5}
foo: Value for foo
after: {'a': 5, 'foo': 'Value for foo'}


In [10]:
# 子类定义__getattr__，容易出现无限递归
class subtest(test):
    def __getattr__(self, name):
        print('call subtest __getattr__')
        return super().__getattr__(name) #  通过此方法来调用父类的方法，避免无限递归

st = subtest()
print(st.b)
print(st.b) # 已存在

call subtest __getattr__
Value for b
Value for b


In [12]:
# __getattribute__：每次访问对象的属性都会调用该函数，可以起到动态监控属性情况
# 而__getattr__只会调用一次（当属性未创建时）
class test:
    def __getattribute__(self, name):
        print(f'called __getattribute__({name})')
        try:
            super().__getattribute__(name) 
        except AttributeError: # 若属性不存在，会报错AttributeError
            value = 'Value for %s'% name
            setattr(self, name, value)
            return value

t = test()
print(t.a)
print(t.a)# 每次访问都调用__getattribute__方法

called __getattribute__(a)
Value for a
called __getattribute__(a)
None


In [15]:
# __setattr__当对对象的属性赋值时，会调用此函数
class test:
    def __setattr__(self, name, value):
        print('call test:__setattr__')
        super().__setattr__(name, value)

class subtest(test):
    def __setattr__(self, name, value):
        print('call subtest:__setattr__')
        super().__setattr__(name, value)

st = subtest()
st.a = 1
print(st.a)
st.b = 2
print(st.b)

call subtest:__setattr__
call test:__setattr__
1
call subtest:__setattr__
call test:__setattr__
2


In [16]:
# 注意无限递归问题
# 在__getattribute__和__setattr__中都要用到super().__XX__来避免无限递归
class test:
    def __init__(self, data):
        self._data = data
    
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data') # 从实例的属性字典里直接取值，避免无限循环
        return data_dict[name]

        # wrong
        # return self._data[name] -》产生无限递归

t = test({'a': '1'})
print(t.a)

1


### staticmethod() 返回函数的静态方法

In [4]:
### staticmethod 方法不强制要求传递参数,不需要实例化
class test:

    def __init__(self):
        pass

    @staticmethod
    def check(data):
        if not (0 <= data <= 1):
            raise ValueError('data must be [0,1]')

test.check(1) # 无需实例化类，也可以实例化后使用
test.check(2)

ValueError: data must be [0,1]

### @property，@func.setter 取代get和set方法

In [5]:
# 编写新类时，应该用简单的public属性来定义其接口，而不要手工实现set和get
# 如果访问对象的某个属性时，需要表现出特殊的行为，那就用@property来定义这种行为
# 例如：防止设置对象的属性小于0

class test:

    def __init__(self):
        self._a = 0
        self._reverse_a = 0

    @property
    def a(self):
        return self._a

    @a.setter
    def a(self, value):
        if value < 0:
            raise ValueError(f'{value} a must be >= 0')
        self._a = value
        self._reverse_a = -self._a # 设置了a则附带设置其他值

t = test()
t.a = 1
print(f'a: {t.a}')
print(f'_reverse_a: {t._reverse_a}')
t.a = -1 # a must be >= 0

a: 1
_reverse_a: -1


ValueError: -1 a must be >= 0

In [13]:
class test1(test):

    def __init__(self):
        super().__init__()

t1 = test1()
t1.a = 1
print(t1.a)
print(t1._reverse_a)
t2 = test1()
t2.a = 2
print(t2.a)
print(t1.a)

1
-1
2
1


### 使用collections.abc，从模块中的抽象基类中继承，编写自制的容器类型

In [25]:
from collections.abc import Sequence

class MySequence(Sequence):

    def __init__(self, data):
        super().__init__()
        self.data = data

    def __getitem__(self, idx):
        return self.data[idx]

    def __len__(self):
        return len(self.data)

m = MySequence([1,2])
print(m[0])
print(len(m))

1
2


### dict.setdefault(key, value) 当key空时幅值value

In [17]:
dict_ = {}
dict_.setdefault('a', 1)
dict_.setdefault('a', 0) # 已经有值，则不会再幅默认值
print(dict_)

{'a': 1}


### public，protected、private

In [14]:
# protected：本质上与 public 属性使用相同，但命名上体现了保护目的
# 可多用protected属性，并在文档中把这些字段的合理用法告诉子类的开发者
# 只有当子类不受自己控制时，才可以考虑用private属性来避免名称冲突

class test:

    def __init__(self):
        pass

    def public_method(self):
        print('public')
        self.public_value = 'public'

    def _protected_method(self):
        print('protected')
        self._protected_value = 'protected'

    def __private_method(self):
        print('private')
        self.__private_value = 'private'

t = test()
t.public_method()
t._protected_method()

t._test__private_method() # private本质只是换了一个名称保存
t.__private_method() # 直接访问无法获得

public
protected
private


AttributeError: 'test' object has no attribute '__private_method'

In [16]:
print(t.public_value)
print(t._protected_value)
print(t._test__private_value)
print(t.__private_value)

public
protected
private


AttributeError: 'test' object has no attribute '__private_value'

### 用super初始化父类，避免钻石继承问题

In [10]:
# 用内置的super函数来初始化父类，避免钻石继承问题
# 注意继承顺序(与实际执行相反的顺序)
class A:
    def __init__(self, value):
        self.value = value
        print('A:', self.value)

class A1(A):
    def __init__(self, value):
        super().__init__(value)
        self.value *= 5
        print('A1:', self.value)

class A2(A):
    def __init__(self, value):
        super().__init__(value)
        self.value += 2
        print('A2:', self.value)


class B(A2, A1): # 顺序与实际调用顺序相反：深度优先-》A,A1,A2
    def __init__(self, value):
        super().__init__(value)

b = B(5)
b.value

A: 5
A1: 25
A2: 27


27

### 以 @classmethod 形式的多态去通用地构建对象 

In [5]:
# 多态：使得继承体系中的多个类都能以各自所独有的方式来实现某个方法。这些类，都满足相同的接口或继承自相同的抽象类，但却有着各自不同的功能
# classmethod 修饰符对应的函数不需要实例化，不需要 self 参数，但第一个参数需要是表示自身类的 cls 参数，可以来调用类的属性，类的方法，实例化对象等

class test:

    def __init__(self, normalization_data):
        self.normalization_data = normalization_data

    @classmethod # 把输入的预操作（具体实现）放在此处，增加通用性
    def normalization(cls, data):
        return cls(data.clamp(0, 1))

import torch
a = torch.randn(size=(3,1))
print(a)
t = test.normalization(a)
print(t.normalization_data)

tensor([[ 0.0279],
        [-1.7643],
        [ 0.0514]])
tensor([[0.0279],
        [0.0000],
        [0.0514]])


### collections模块中的namedtuple，定义精简而不可变的数据类

In [54]:
# 既可以按位置指定其中各项，也可以采用关键字来指定
# 缺点：无法指定各参数的默认值
import collections
Grade = collections.namedtuple('Grade', ('score', 'weight'))
print(Grade(90, 0.3))

Grade(score=90, weight=0.3)


### 在函数参数部分加上*，要求只能以关键字形式指定参数

In [53]:
def test(a, *, b=1):
    print(a, b)

test(1)
test(1, b=2)
test(1, 2) # 必须以关键字形式指定参数

1 1
1 2


TypeError: test() takes 1 positional argument but 2 were given

### 用None来做动态默认值

In [47]:
# 如果参数实际默认值是可变类型（mutable），记得用None作为形式上的默认值
# 错误代码：用{}作为默认参数值
import json
from datetime import datetime
def decode(data, default={}, time=datetime.now()):
    print(time)
    try:
        return json.load(data)
    except AttributeError:
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('foo:', foo)
print('var:', bar) # 同一个对象，在函数建立的时候就建立

2022-05-18 17:41:27.968258
2022-05-18 17:41:27.968258
foo: {'stuff': 5, 'meep': 1}
var: {'stuff': 5, 'meep': 1}


In [48]:
# 使用None为参数默认值
def decode(data, default=None, time=None):
    time = datetime.now()
    default = {}
    print(time)
    try:
        return json.load(data)
    except AttributeError:
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('foo:', foo)
print('var:', bar) 

2022-05-18 17:42:20.413731
2022-05-18 17:42:20.414040
foo: {'stuff': 5}
var: {'meep': 1}


### 函数接受可选的位置参数：*args

In [42]:
# 函数参数添加*args, 会自动有默认值，会将输入组成元组（有位置顺序，所以叫位置参数）
# 要扩展*args位置参数，就必须修改旧代码，可以用关键字形式指定的参数
def test(a, *args):
    if not args:
        print(a)
    else:
        str_ = ', '.join(str(x) for x in args)
        print(a, str_)

test('it is', 1)
test('it is')

test('it is', [1,2])
test('it is', *[1,2]) # *输入列表，会被作为位置参数输入

it is 1
it is
it is [1, 2]
it is 1, 2


### 迭代器iter，生成器

In [24]:
# iter 会返回迭代器对象，迭代器对象中实现了__next__方法，可用next函数来取值
# for x in a: 会调用iter(a)
a = [1,2,3,4,5,6,7,8]
print(f'iter: {iter(a)}')
print(iter(a).__next__())
print(next(iter(a)))

iter: <list_iterator object at 0x7fbc72248280>
1
1


In [36]:
# 迭代器协议约定 iter：
# 以确保调用者传进来的参数，并不是迭代器对象本身
# 如果迭代器对象传给内置的iter函数，那么此函数会把迭代器返回
# 反之，如果是个容器类型的对象，那么每次都会返回新的迭代器对象
a = [1,2,3,4,5,6,7,8]
it1 = iter(a)
it2 = iter(it1)
assert it1 is it2
print(list(it1))
print(list(it2)) # it1与it2同一个对象，it1在之前的list中已释放完

it1 = iter(a)
it3 = iter(a)
try:
    assert it1 is it3
except AssertionError as e:
    print(e)

print(list(it1))
print(list(it3)) # it1与it3不是同一个迭代器对象，但值相同

[1, 2, 3, 4, 5, 6, 7, 8]
[]

[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8]


### 生成器 generator，next

In [19]:
# 生成器generator使用yield表达式的函数，不会真的运行，而是返回迭代器
# 每次在迭代器上面的调用next，迭代器会把生成器推进到下一个yield表达式
# 并把yield表达式的结果返回

a = [1,2,3,4,5,6,7,8]

def generator(data):
    for x in data:
        yield x

it = generator(a)
print(f'it: {it}, type: {type(it)}')
print(f'next: {next(it)}')
print(f'next: {next(it)}')

def generator(data):

    if data[0] == 1:
        yield float('inf')

    for x in data:
        yield x
it = generator(a)
print(f'it: {it}, type: {type(it)}')
print(f'next: {next(it)}')
print(f'next: {next(it)}')

# 迭代器只能使用一次
b = list(it)
print(f'b: {b}')
c = list(it)
print(f'c: {c}')

it: <generator object generator at 0x7fbc72234d60>, type: <class 'generator'>
next: 1
next: 2
it: <generator object generator at 0x7fbd5b91bd60>, type: <class 'generator'>
next: inf
next: 1
b: [2, 3, 4, 5, 6, 7, 8]
c: []


In [16]:
import torch
import torchvision
import torchvision.transforms as transforms

mnist = torchvision.datasets.MNIST(r'/data/lzh/data/datasets/mnist')
print(f'mnist: {mnist}, type: {type(mnist)}')
print(dir(mnist)) # 获得全部成员

def load(data):
    for x, y in mnist:
        x = transforms.ToTensor()(x)
        yield x, y

loader = load(mnist)
x, y = next(loader)
print(x.shape, y)

mnist: Dataset MNIST
    Number of datapoints: 60000
    Root location: /data/lzh/data/datasets/mnist
    Split: Train, type: <class 'torchvision.datasets.mnist.MNIST'>
['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_check_exists', '_check_legacy_exist', '_format_transform_repr', '_is_protocol', '_load_data', '_load_legacy_data', '_repr_indent', 'class_to_idx', 'classes', 'data', 'download', 'extra_repr', 'functions', 'mirrors', 'processed_folder', 'raw_folder', 'register_datapipe_as_function', 'register_function', 'resources', 'root', 'target_transform'

### 闭包，nonlocal，global

In [5]:
# 作用域bug：防止函数中的局部变量污染函数外面的那个模块

def test():
    ex_a = 1
    def closure():
        ex_a = 2 # 此处会创建一个新的变量
        print(ex_a)
    closure()
    print(ex_a)

test()

2
1


In [3]:
def test(a):
    
    print(id(a))
    def closure():
        a += 1 # 不能修改
        print(id(a))
        b = a
        assert b is a
        print(id(b))
        b += 1
        print(b)
        return a
    
    return closure

closure = test(a=1)
closure()


94184960949728


UnboundLocalError: local variable 'a' referenced before assignment

In [6]:
# 用 nonlocal 能够获得闭包内的数据，从上层作用域中查找变量
# nonlocal的限制：不能延伸到模块级别，防止污染全局作用域
# 最好仅在简单的函数中使用
def test():
    ex_a = 1
    def closure():
        nonlocal ex_a
        ex_a = 2 # 会修改闭包外的变量
        print(ex_a)
    closure()
    print(ex_a)

test()

2
2


In [17]:
# global 将会直接修改模块作用域里的那个变量
ex_a = 2

def test():
    global ex_a
    ex_a += 3
    print(ex_a)
test()
print(ex_a)

5
5


### 合理利用try/except/else/finally结构中的每个代码块

In [1]:
# try/finally：既要将异常向上传播，又要在异常发生时执行清理工作
# try/except/else：选择哪些异常由自己的代码处理，哪些异常会传播到上一级，如果无异常，则执行else

a = 1
try:
    a /= 0
finally:
    print(a)

1


ZeroDivisionError: division by zero

### 了解bytes、str与unicode区别

In [2]:
# python 3有两种表示字符序列的类型：bytes和str。
# bytes：该的实例包含原始的8位值（原始的字节，由于每个字节有8个二进制位，所以是原始的8位数，也叫原生8位值，纯8位值）
# str：该的实例包含unicode字符

In [14]:
# unicode字符：表示为二进制数据（原始8位值）有很多种方法，最常见的编码方式就是UTF-8
#   把unicode字符转换成二进制数据：encode
#   把二进制数据转换成unicode字符串：decode
str_ = '你'
bytes_ = str_.encode()
print(f'bytes: {bytes_}, type: {type(bytes_)}')

str__ = bytes_.decode()
print(f'str: {str__}, type: {type(str__)}')

bytes: b'\xe4\xbd\xa0', type: <class 'bytes'>
str: 你, type: <class 'str'>


In [48]:
# 写入和读出二进制数据到文件中
# 读取:
import os
with open('./random.bin', 'wb') as f:
    bytes_ = 'a'.encode()
    f.write(bytes_)

with open('./random.bin', 'rb') as f:
    bytes_ = f.readline()
    print(bytes_)
    print(bytes_.decode())

b'a'
a
